In [2]:
# import numpy and pandas, and DataFrame / Series
import math
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import scipy.optimize as optimize
import scipy
# Set some numpy options
np.set_printoptions(suppress=True)

# Set some pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# And some items for matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
#pd.options.display.mpl_style = 'default'

# read http://scikit-image.org/docs/dev/user_guide.html 
# for documentation
from skimage import io
from skimage import color

# sudo /usr/lib/anaconda/bin/conda install -c https://conda.binstar.org/menpo opencv3
#import cv2

In [3]:
TRAIN = pd.read_csv("kaggle/sf_crime/train.csv")

In [4]:
print (TRAIN.head(5))

                 Dates        Category                      Descript  \
0  2015-05-13 23:53:00        WARRANTS                WARRANT ARREST   
1  2015-05-13 23:53:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
2  2015-05-13 23:33:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
3  2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   
4  2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   

   DayOfWeek PdDistrict      Resolution                    Address  \
0  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
1  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
2  Wednesday   NORTHERN  ARREST, BOOKED  VANNESS AV / GREENWICH ST   
3  Wednesday   NORTHERN            NONE   1500 Block of LOMBARD ST   
4  Wednesday       PARK            NONE  100 Block of BRODERICK ST   

            X          Y  
0 -122.425892  37.774599  
1 -122.425892  37.774599  
2 -122.424363  37.800414  
3 -122.426995  37.800873  
4 -122.4387

In [5]:
TRAIN.Dates.apply(lambda x: x.split(" ")[1].split(":")[0])

0         23
1         23
2         23
3         23
4         23
5         23
6         23
7         23
8         23
9         23
10        22
11        22
12        22
13        22
14        22
15        22
16        22
17        21
18        21
19        21
20        21
21        21
22        21
23        21
24        21
25        21
26        21
27        21
28        21
29        20
30        20
31        20
32        20
33        20
34        20
35        20
36        20
37        20
38        20
39        20
40        20
41        20
42        20
43        20
44        20
45        20
46        20
47        20
48        19
49        19
50        19
51        19
52        19
53        19
54        19
55        19
56        19
57        19
58        19
59        19
60        19
61        19
62        19
63        19
64        19
65        19
66        19
67        19
68        19
69        19
70        19
71        19
72        19
73        19
74        19
75        18
76        18

In [6]:
TRAIN.X * TRAIN.Y

0        -4624.588916
1        -4624.588916
2        -4627.691645
3        -4627.847257
4        -4624.699819
5        -4616.246576
6        -4618.436910
7        -4616.770092
8        -4627.943197
9        -4628.396573
10       -4628.396573
11       -4622.410674
12       -4625.108713
13       -4626.048506
14       -4618.058904
15       -4619.150804
16       -4625.229259
17       -4620.539760
18       -4618.738362
19       -4625.047722
20       -4621.566738
21       -4621.038611
22       -4625.047722
23       -4625.047722
24       -4628.063220
25       -4625.408794
26       -4626.348372
27       -4619.357528
28       -4622.838159
29       -4624.185371
30       -4625.815845
31       -4623.366720
32       -4625.979503
33       -4626.173623
34       -4626.173623
35       -4623.734416
36       -4627.389607
37       -4626.567638
38       -4619.880065
39       -4617.029500
40       -4625.244077
41       -4624.255067
42       -4625.250224
43       -4618.152554
44       -4618.152554
45       -

In [27]:
categories = TRAIN["Category"].unique()
np.ndarray.sort(categories)
num_categories = categories.shape[0]
print(categories.shape[0])
print(categories)
num_labels = num_categories

39
['ARSON' 'ASSAULT' 'BAD CHECKS' 'BRIBERY' 'BURGLARY' 'DISORDERLY CONDUCT'
 'DRIVING UNDER THE INFLUENCE' 'DRUG/NARCOTIC' 'DRUNKENNESS' 'EMBEZZLEMENT'
 'EXTORTION' 'FAMILY OFFENSES' 'FORGERY/COUNTERFEITING' 'FRAUD' 'GAMBLING'
 'KIDNAPPING' 'LARCENY/THEFT' 'LIQUOR LAWS' 'LOITERING' 'MISSING PERSON'
 'NON-CRIMINAL' 'OTHER OFFENSES' 'PORNOGRAPHY/OBSCENE MAT' 'PROSTITUTION'
 'RECOVERED VEHICLE' 'ROBBERY' 'RUNAWAY' 'SECONDARY CODES'
 'SEX OFFENSES FORCIBLE' 'SEX OFFENSES NON FORCIBLE' 'STOLEN PROPERTY'
 'SUICIDE' 'SUSPICIOUS OCC' 'TREA' 'TRESPASS' 'VANDALISM' 'VEHICLE THEFT'
 'WARRANTS' 'WEAPON LAWS']


In [8]:
m = TRAIN.shape[0]
y = np.ones((m,1))*-1
for i in np.arange(categories.shape[0]):
    have_true = TRAIN["Category"]==categories[i]
    for j in np.arange(m):
        if(have_true[j] == True):
            y[j] = i

In [9]:
y[1:10]

array([[ 21.],
       [ 21.],
       [ 16.],
       [ 16.],
       [ 16.],
       [ 36.],
       [ 36.],
       [ 16.],
       [ 16.]])

In [10]:
dayOfWeek = TRAIN["DayOfWeek"].unique()
np.ndarray.sort(dayOfWeek)
num_days = dayOfWeek.shape[0]
print(dayOfWeek.shape[0])
print(dayOfWeek)

7
['Friday' 'Monday' 'Saturday' 'Sunday' 'Thursday' 'Tuesday' 'Wednesday']


In [18]:
addresses = TRAIN["Address"].unique()
np.ndarray.sort(addresses)
num_addresses = dayOfWeek.shape[0]
print(addresses.shape[0])
#print(addresses)

23228


In [25]:
X = np.ones((m,6))

In [26]:
for i in np.arange(m):
    X[i, 0] = TRAIN.iloc[i].Dates.split(" ")[1].split(":")[0] # time
    X[i, 1] = np.argmax(TRAIN.iloc[i].DayOfWeek==dayOfWeek) # Day of Week
    X[i, 2] = np.argmax(TRAIN.iloc[i].Address==addresses) # Address
    X[i, 3] = TRAIN.iloc[i].X
    X[i, 4] = TRAIN.iloc[i].Y
    X[i, 5] = TRAIN.iloc[i].X * TRAIN.iloc[i].Y

    
X[0]

array([    23.        ,      6.        ,  19790.        ,   -122.42589168,
           37.7745986 ,  -4624.58891575])

In [28]:
input_layer_size  = 6 #img_size*img_size;  # 32x32 Input Images of Digits
hidden_layer_size = 25;   # 155 hidden units

In [29]:
def sigmoid(z):
#SIGMOID Compute sigmoid functoon
#   J = SIGMOID(z) computes the sigmoid of z.
    
    g = 1.0 / (1.0 + np.exp(-z));
    return g

def sigmoidGradient(z):
#SIGMOID Compute sigmoid functoon
#   J = SIGMOID(z) computes the sigmoid of z.
    
    g  = sigmoid(z) * (1-sigmoid(z))
    return g

In [30]:
# See Lecture 51
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lamda):
    # %NNCOSTFUNCTION Implements the neural network cost function for a two layer
    # %neural network which performs classification
    # %   [J grad] = NNCOSTFUNCTON(nn_params, hidden_layer_size, num_labels, ...
    # %   X, y, lambda) computes the cost and gradient of the neural network. The
    # %   parameters for the neural network are "unrolled" into the vector
    # %   nn_params and need to be converted back into the weight matrices. 
    # % 
    # %   The returned parameter grad should be a "unrolled" vector of the
    # %   partial derivatives of the neural network.
    # %

    # % Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # % for our 2 layer neural network
    
    Theta1 = nn_params[0:hidden_layer_size*(input_layer_size+1)].reshape((hidden_layer_size, 
                                                                          (input_layer_size+1)), 
                                                                         order='F')
    # print(Theta1.shape)
    
    
    Theta2 = nn_params[hidden_layer_size*(input_layer_size+1):].reshape((num_labels, 
                                                                         (hidden_layer_size+1)), 
                                                                        order='F')
    # print(Theta2.shape)
    
    # Setup some useful variables
    m = X.shape[0]
    
    # You need to return the following variables correctly 
    J = 0;
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    # % ====================== YOUR CODE HERE ======================
    # % Instructions: You should complete the code by working through the
    # %               following parts.
    # %
    # % Part 1: Feedforward the neural network and return the cost in the
    # %         variable J. After implementing Part 1, you can verify that your
    # %         cost function computation is correct by verifying the cost
    # %         computed in ex4.m
    # %
    # % Part 2: Implement the backpropagation algorithm to compute the gradients
    # %         Theta1_grad and Theta2_grad. You should return the partial derivatives of
    # %         the cost function with respect to Theta1 and Theta2 in Theta1_grad and
    # %         Theta2_grad, respectively. After implementing Part 2, you can check
    # %         that your implementation is correct by running checkNNGradients
    # %
    # %         Note: The vector y passed into the function is a vector of labels
    # %               containing values from 1..K. You need to map this vector into a 
    # %               binary vector of 1's and 0's to be used with the neural network
    # %               cost function.
    # %
    # %         Hint: We recommend implementing backpropagation using a for-loop
    # %               over the training examples if you are implementing it for the 
    # %               first time.
    # %
    # % Part 3: Implement regularization with the cost function and gradients.
    # %
    # %         Hint: You can implement this around the code for
    # %               backpropagation. That is, you can compute the gradients for
    # %               the regularization separately and then add them to Theta1_grad
    # %               and Theta2_grad from Part 2.
    # %    
    
    a1 = np.hstack((np.ones((m,1)), X)) # (5000 x 401)
    z2 = a1.dot(Theta1.T)
    a2 = sigmoid(z2) # (5000 x 401) x (401 x 25) = 5000 x 25
    a2 = np.hstack((np.ones((a2.shape[0], 1)), a2)) # 5000 x 26    
    z3 = a2.dot(Theta2.T)
    a3 = sigmoid(z3) # (5000 x 26) x (26 x 10) = 5000 x 10
    h = a3
    
    # Explode y into 10 values with Y[i] := i == y.
    Y = np.zeros((m, num_labels)) # 5000 x 10
    for i in np.arange(m):
        Y[i, int(np.mod(y[i,0],num_labels))] = 1
    
    tmp = 1/m*np.sum(np.sum(-(Y*np.nan_to_num(np.log(h)))-(1-Y)*np.nan_to_num(np.log(1-h))))
    
    J_int = tmp + (lamda/(2*m))*np.sum(np.sum(Theta1[:, 1:] * Theta1[:, 1:])) 
    J = J_int + (lamda/(2*m))*np.sum(np.sum(Theta2[:, 1:] * Theta2[:, 1:])) 
    
    # 2. Backpropagate to get gradient information.
    s3 = h - Y;  # 5000 x 10
    s2 = sigmoidGradient(np.hstack((np.ones((z2.shape[0], 1)), z2))) # (5000 x 401) x (401 x 25) = 5000 x 25 +1 
    s2 = s3.dot(Theta2) * s2 # 5000 x 26

    # Vectorized ftw:
    Theta2_grad = (1/m) * s3.T.dot(a2) # ?? 10 x 26
    Theta1_grad = (1/m) * s2[:,1:].T.dot(a1) # ?? 25 x 5000 x 500 x 401 = 25 x 401
    
    p2 = (lamda/m) * np.hstack((np.zeros((Theta2.shape[0],1)), Theta2[:, 1:])) # (5000 x 401)    
    p1 = (lamda/m) * np.hstack((np.zeros((Theta1.shape[0],1)), Theta1[:, 1:])) # (5000 x 401)
 
    Grad2 = Theta2_grad + p2
    
    Grad1 = Theta1_grad + p1
    
    #  % Unroll gradients
    grad = np.hstack((Grad1.flatten(order='F'), Grad2.flatten(order='F')))
    
    return (J, grad)

In [31]:
def randInitializeWeights(L_in, L_out):
# %RANDINITIALIZEWEIGHTS Randomly initialize the weights of a layer with L_in
# %incoming connections and L_out outgoing connections
# %   W = RANDINITIALIZEWEIGHTS(L_in, L_out) randomly initializes the weights 
# %   of a layer with L_in incoming connections and L_out outgoing 
# %   connections. 
# %
# %   Note that W should be set to a matrix of size(L_out, 1 + L_in) as
# %   the first row of W handles the "bias" terms
# %
    
    epsilon_init = 0.12;
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init;
    
    return W


In [32]:
# %% ================ Part 6: Initializing Pameters ================
# %  In this part of the exercise, you will be starting to implment a two
# %  layer neural network that classifies digits. You will start by
# %  implementing a function to initialize the weights of the neural network
# %  (randInitializeWeights.m)

print('\nInitializing Neural Network Parameters ...\n')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size);
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels);

# % Unroll parameters
initial_nn_params = np.hstack((initial_Theta1.flatten(), initial_Theta2.flatten()))


Initializing Neural Network Parameters ...



In [33]:
#%% =================== Part 8: Training NN ===================
#%  You have now implemented all the code necessary to train a neural
#%  network. To train your neural network, we will now use "fmincg", which
#%  is a function which works similarly to "fminunc". Recall that these
#%  advanced optimizers are able to train our cost functions efficiently as
#%  long as we provide them with the gradient computations.
#%
print('\nTraining Neural Network... \n')

# %  After you have completed the assignment, change the MaxIter to a larger
# %  value to see how more training helps.
# options = optimset('MaxIter', 50);

# %  You should also try different values of lambda
lamda = 1;
 
options = {'maxiter': 1000}
# Try 'CG', 'BFGS', 'L-BFGS-B'
Result = optimize.minimize(fun = nnCostFunction, 
                           x0 = initial_nn_params, 
                           args = (input_layer_size, hidden_layer_size, num_labels, X, y, lamda), 
                           method = 'L-BFGS-B',
                           #jac = lrCostGradFunction, options=options)
                           jac=True, options=options)

nn_params = Result.x

if Result.success != True:
    print(Result.message)

# Obtain Theta1 and Theta2 back from nn_params
Theta1 = nn_params[0:hidden_layer_size*(input_layer_size+1)].reshape((hidden_layer_size, 
                                                                      (input_layer_size+1)), 
                                                                     order='F')
# print(Theta1.shape)
    
    
Theta2 = nn_params[hidden_layer_size*(input_layer_size+1):].reshape((num_labels, 
                                                                     (hidden_layer_size+1)), 
                                                                    order='F')

print('Program paused. Press enter to continue.\n');


Training Neural Network... 

Program paused. Press enter to continue.



In [34]:
def predict(Theta1, Theta2, X):
#%PREDICT Predict the label of an input given a trained neural network
#%   p = PREDICT(Theta1, Theta2, X) outputs the predicted label of X given the
#%   trained weights of a neural network (Theta1, Theta2)

#% Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

#% You need to return the following variables correctly 
    p = np.zeros((m, 1))

    h1 = sigmoid(np.hstack((np.ones((m, 1)), X)).dot(Theta1.T))
    h2 = sigmoid(np.hstack((np.ones((m, 1)), h1)).dot(Theta2.T))
    
    
    p = np.argmax(h2, axis=1)

#% =========================================================================
    return p

In [35]:
# %% ================= Part 10: Implement Predict =================
# %  After training the neural network, we would like to use it to predict
# %  the labels. You will now implement the "predict" function to use the
# %  neural network to predict the labels of the training set. This lets
# %  you compute the training set accuracy.

pred = predict(Theta1, Theta2, X);

print("\nTraining Set Accuracy: {0}\n".format(np.mean(pred == (np.mod(y[:,0],num_labels))) * 100)) 


Training Set Accuracy: 19.919161686876244



## Write kaggle submission file

In [ ]:
import glob

files_a = None
for file in glob.glob("../../02-logistic-regression/jupyter/kaggle/test/*.Bmp"):
    n_s= file.split('/')[6].split('.')[0]
    n = int(n_s)
    if(files_a is None):
        files_a = np.array(n)
    else:
        files_a = np.vstack((files_a, n))
        
np.ndarray.sort(files_a)       
files_a = files_a.flatten().tolist()
print(len(files_a))

In [ ]:
#TEST = None
#img_size = 32
#for i in files_a:
#    img = color.rgb2gray (io.imread("../../02-logistic-regression/jupyter/kaggle/test/{0}.Bmp".format(i)))
#    img32 = scipy.misc.imresize(img, (img_size,img_size))
#    img32[img32<128] = 0
#    img32[img32>=128] = 1
#    img_array = np.array(img32).flatten()
#    img_array = img_array.reshape(1, img_array.shape[0])
#    if(TEST is None):
#        TEST = img_array
#    else:
#        TEST = np.vstack((TEST, img_array))

TEST = None
for i in files_a:
    img_array = read_image("../../02-logistic-regression/jupyter/kaggle/test/{0}.Bmp".format(i), img_size)
    if(TEST is None):
        TEST = img_array
    else:
        TEST = np.vstack((TEST, img_array))

In [ ]:
test_pred = predict(Theta1, Theta2, TEST);

In [ ]:
print(TEST.shape)
print(test_pred.shape)
print(len(files_a))

In [ ]:
d = {'Class' : test_pred.flatten(), 'ID' : files_a}

df = pd.DataFrame(d)

In [ ]:
y_test = []

for i in files_a:
    y_test.append(labels_s[df[df["ID"]==i].Class.values[0]])
f = open("kaggle/test/submission.csv", 'w')

print(len(y_test))
j = 0
f.write("ID,Class\n")
for i in files_a:
    f.write("{0},{1}\n".format(i, y_test[j]))
    j = j+1
    
f.close()

f = open("kaggle/test/submission2.csv", 'w')
j = 0
f.write("ID,Class\n")
for i in files_a:
    f.write("{0},{1}\n".format(files_a[j], labels_s[test_pred[j]]))
    j = j+1
    
f.close()   

## Sanity check for individual file

In [ ]:
i = 9928

img_array_bw = read_image_bw("../../02-logistic-regression/jupyter/kaggle/test/{0}.Bmp".format(i), img_size)
img_array = read_image("../../02-logistic-regression/jupyter/kaggle/test/{0}.Bmp".format(i), img_size)

l_pred = predict(Theta1, Theta2, img_array);

print(l_pred)
print(labels_s[l_pred[0]])

SAN = None

SAN = np.vstack((img_array_bw, img_array))
displayData(SAN);